In [1]:
from util.ml_data import MLData
from util.modify_inp import qchem_out_aimd_multi
import numpy as np
multi = qchem_out_aimd_multi()
multi.read_files([
    "./examples/aimd_bodipy_nvt_1.out",
    "./examples/aimd_bodipy_nvt_2.out"
])



# 转换成 kcal/mol, Angstrom, kcal/mol/Angstrom
multi.export_numpy(prefix="./examples/full_",
                   energy_unit="kcal",
                   distance_unit="ang",
                   force_unit=("kcal", "ang"))




(array([[[ 18.63732158,  -6.43775701, -18.23080281],
         [ 18.01188809,  -6.44869997, -19.47629995],
         [ 17.89140931,  -7.63713402, -20.20409137],
         ...,
         [ 16.99586827,  -4.84456373, -20.6108839 ],
         [ 18.99356186,  -4.95728415, -16.76380659],
         [ 17.72282137,  -3.30427411, -18.5098384 ]],
 
        [[ 18.6374422 ,  -6.43722695, -18.22965803],
         [ 18.00688045,  -6.44945552, -19.48057692],
         [ 17.89390196,  -7.63824366, -20.20137821],
         ...,
         [ 16.98457613,  -4.86036314, -20.60021578],
         [ 18.98529201,  -4.9543911 , -16.7488636 ],
         [ 17.71658536,  -3.3169891 , -18.4985557 ]],
 
        [[ 18.63733354,  -6.43684381, -18.22851656],
         [ 18.00192865,  -6.44993574, -19.48455093],
         [ 17.89625342,  -7.63924804, -20.1988349 ],
         ...,
         [ 16.97152698,  -4.87433444, -20.59252998],
         [ 18.97702628,  -4.95182831, -16.73527423],
         [ 17.71055158,  -3.32815659, -18.48785082]

In [2]:
# 1. 读取 (可扩展字段)
dataset = MLData(prefix="./examples/full_", files=["coord","energy","grad","type"])

# 2. 保存划分
dataset.save_split(1000, 200, 200, prefix="./examples/1000_")

# 3. 加载 split 并切片
split = np.load("./examples/1000_split.npz", allow_pickle=True)
train_data = dataset.get_split_data(split, part="idx_train")
val_data   = dataset.get_split_data(split, part="idx_val")


Loaded dataset with 20000 frames, 31 atoms
Saved split to ./examples/1000_split.npz (idx_train=1000, idx_val=200, idx_test=200)


In [5]:
split["idx_train"]

array([18874, 14938,  6524,   965, 16306,  6049, 13820,  4456,  8717,
       19982, 17508,  8641,  2216, 12228,  3929,    13,  1367,  6808,
       10253,  2007, 16767,  6388, 18503, 12134,  4679, 10107, 19027,
          85,  3448, 18531, 16950, 18676, 19043, 17347,  5853, 19338,
       10151,  9941,  9369, 13555, 12757, 14904,  6755, 12132, 12405,
       19746,  8959, 17893,  6772,   114, 11491, 19496,  1895, 17780,
        9679,  6931,  6997, 19565,   481, 11482, 12448,  1261,  3555,
       17013,  3338,  2580,  6263,  3439, 18323,  1473,  5977, 10375,
       17960,  6555,  3413,  4997,  6924,  6350,  3092, 19533,  6310,
        8970,  3242, 15662, 13754,  9411, 15367,  5858,   341, 10367,
       14019, 10723, 13800, 18670, 16481,  1772,  7104,   676,  1195,
        1909, 11013, 10338, 18762,    74, 16358,  6167,  9383, 18178,
         411, 18551, 12595, 13422,  7667,  3281,  9791, 10533,  6824,
       10815, 11729, 13228, 16190, 15802, 13591,  1549,  6036,  7127,
        3282,  9218,

In [2]:
e = np.load("./examples/full_energy.npy", allow_pickle=True)

In [3]:
e.max()

np.float64(-571880.1151127727)

In [4]:
e.min()

np.float64(-571922.4940068978)